# Run all 

In [ ]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper

nr_ads_pre = len(load_local_data())

scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
#scrapers = [CinodeScraper()]
#scrapers = []
for s in scrapers:
    response = s.request_status()
    scraped_payload_dict = s.scrape_jobs_payloads_dict(response)

    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_payload_dict = dict(zip(old_payload_data['id'], old_payload_data['raw_payload']))
    old_bronze_data = load_local_data()
    
    new_payload_dict = s.return_new_payloads(new_dict=scraped_payload_dict, old_dict=old_payload_dict)
    new_bronze_data = s.parse_bronze_data(new_payload_dict)

    updated_payload_dict = s.concat_dicts(new_payload_dict, old_payload_dict)
    updated_payload_data = pd.DataFrame(list(updated_payload_dict.items()), columns=['id','raw_payload'])
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)

    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
new_bronze_data


Cinode > Response: 200
Cinode > Nmr of scraped adds: 20
Cinode > Parsing bronze data: 20
Total number of new added jobs: 0


,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new
0,Cinode-https://cinode.market/requests/17317,Cinode,https://cinode.market/requests/17317,Software Developer,None,None,Stockholm(Onsite),None,https://cinode.market/requests/17317,2025-09-23 17:42:46,True
1,Cinode-https://cinode.market/requests/17313,Cinode,https://cinode.market/requests/17313,Webutvecklings-team på distans,None,None,Linköping(100% remote),None,https://cinode.market/requests/17313,2025-09-23 17:42:46,True
2,Cinode-https://cinode.market/requests/17312,Cinode,https://cinode.market/requests/17312,Arkivarie e-dokument på 100% distans,None,None,Stockholm(100% remote),None,https://cinode.market/requests/17312,2025-09-23 17:42:46,True
3,Cinode-https://cinode.market/requests/17310,Cinode,https://cinode.market/requests/17310,Senior testare,None,None,Sundsvall(40% remote),None,https://cinode.market/requests/17310,2025-09-23 17:42:46,True
4,Cinode-https://cinode.market/requests/17309,Cinode,https://cinode.market/requests/17309,Lösningsarkitekt på 50% distans till Stockholm,None,None,Stockholm(50% remote),None,https://cinode.market/requests/17309,2025-09-23 17:42:46,True
5,Cinode-https://cinode.market/requests/17308,Cinode,https://cinode.market/requests/17308,Projektledare till långt uppdrag i Stockholm,None,None,Stockholm(40% remote),None,https://cinode.market/requests/17308,2025-09-23 17:42:46,True
6,Cinode-https://cinode.market/requests/17307,Cinode,https://cinode.market/requests/17307,Webbteam (extern webb) - AB Östgötatrafiken - ...,None,None,Linköping(95% remote),None,https://cinode.market/requests/17307,2025-09-23 17:42:46,True
7,Cinode-https://cinode.market/requests/17306,Cinode,https://cinode.market/requests/17306,Testledare Windows uppgradering,None,None,Stockholm(Onsite),None,https://cinode.market/requests/17306,2025-09-23 17:42:46,True
8,Cinode-https://cinode.market/requests/17305,Cinode,https://cinode.market/requests/17305,Frontend-utecklare,None,None,Huskvarna(Onsite),None,https://cinode.market/requests/17305,2025-09-23 17:42:46,True
9,Cinode-https://cinode.market/requests/17303,Cinode,https://cinode.market/requests/17303,Systemutvecklare,None,None,Huskvarna(Onsite),None,https://cinode.market/requests/17303,2025-09-23 17:42:46,True


In [28]:
def write_html(html,name='one_job'):
    with open(f'{name}.html', "w", encoding="utf-8") as f:
        f.write(str(html))



In [77]:
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup

import re
import requests
from bs4 import BeautifulSoup
import re
import requests


class ProfinderScraper:

    def clean_title(self, title: str):
        """Remove 'ID:xxx' from title."""
        return re.sub(r"\s*ID:\d+", "", title).strip()

    def extract_id(self, title: str):
        """Extract job ID if present."""
        match = re.search(r"ID:(\d+)", title)
        return match.group(1) if match else None

    def extract_location(self, title: str):
        """Try to extract location if mentioned with 'i <Ort>'."""
        match = re.search(r" i ([A-ZÅÄÖa-zåäö\s-]+)", title)
        return match.group(1).strip() if match else None


    def scrape_all_jobs(self, max_pages: int = 5):
        all_jobs = []
        for page in range(1, max_pages + 1):
            data = self.get_page(page)
            posts = data.get("postFeedPage", {}).get("posts", {}).get("posts", [])
            if not posts:
                break
            for post in posts:
                title = post.get("title", "")
                job = {
                    "id": self.extract_id(title),
                    "title": self.clean_title(title),
                    "location": self.extract_location(title),
                    "excerpt": post.get("excerpt"),
                    "url": post.get("link"),
                    "published": post.get("firstPublishedDate"),
                }
                all_jobs.append(job)
            print(f"{self.__class__.site} > Scraped {len(posts)} jobs from page {page}")
        return all_jobs





## Load bronze table 

In [30]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
